In [1]:
# # Check if the specified packages are installed and their versions
# import pkg_resources

# # List of required packages and their versions
# required_packages = {
#     "requests": "2.31.0",
#     "pandas": "2.0.3",
#     "pyarrow": "12.0.1",
#     "openai": "1.3.5",
#     "python-decouple": "3.8"
# }

# # Function to check if each package is installed and if the version matches
# def check_packages(packages):
#     installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
#     package_status = {}
#     for package, version in packages.items():
#         if package in installed_packages:
#             installed_version = installed_packages[package]
#             package_status[package] = (installed_version == version, installed_version)
#         else:
#             package_status[package] = (False, None)
#     return package_status

# # Check the packages
# check_packages(required_packages)


{'requests': (True, '2.31.0'),
 'pandas': (True, '2.0.3'),
 'pyarrow': (True, '12.0.1'),
 'openai': (True, '1.3.5'),
 'python-decouple': (True, '3.8')}

In [2]:
# pip install requests==2.31.0

In [3]:
# pip install pandas==2.0.3

In [4]:
# pip install pyarrow==12.0.1

In [5]:
# pip install openai==1.3.5

In [6]:
# pip install python-decouple==3.8

In [1]:
import sys, json
import pandas as pd
from pathlib import Path
from IPython.display import HTML, display
from decouple import config
sys.path.append("../src/")
from llm_helpers import chatgpt_ask

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

data_dir = Path(".").absolute().parent/"data"
ls = lambda p:print("\n".join(map(str,p.iterdir())))

ls(data_dir)

/Users/davidgerman/Downloads/llm_workshop/data/sample_apps.parquet


In [8]:
df = pd.read_parquet(data_dir / "sample_apps.parquet").sample(9)
df

,bundle_id,title,description,store_url,category_names,ios
36869,com.nexters.herowars,Hero Wars – Fantasy Battles,Unlock skills and battle enemies with Hero War...,https://play.google.com/store/apps/details?id=...,"GAME_ROLE_PLAYING,GAME",False
1021,1105855019,Gardenscapes,Welcome to Gardenscapes—the first hit from Pla...,https://apps.apple.com/us/app/gardenscapes/id1...,"Games,Entertainment,Puzzle,Simulation",True
2156,1200391796,June's Journey: Hidden Objects,I spy a mystery! Find the hidden objects to cr...,https://apps.apple.com/us/app/junes-journey-hi...,"Games,Adventure,Puzzle",True
60188,net.supertreat.solitaire,Solitaire Grand Harvest,Welcome to Solitaire Grand Harvest! Play this ...,https://play.google.com/store/apps/details?id=...,"GAME_CARD,GAME",False
659,air.com.playtika.slotomania,Slotomania™ Slots Casino Games,Slotomania wishes you a Happy New Year! Play n...,https://play.google.com/store/apps/details?id=...,"GAME_CASINO,GAME",False
34033,com.mistplay.mistplay,MISTPLAY: Play to earn rewards,"Introducing Mistplay, the app that lets you ea...",https://play.google.com/store/apps/details?id=...,"ENTERTAINMENT,APPLICATION",False
6673,892521917,Tiki Solitaire TriPeaks,Tiki Solitaire TriPeaks: the classic Solitaire...,https://apps.apple.com/us/app/tiki-solitaire-t...,"Games,Card,Puzzle",True
26290,com.hwqgrhhjfd.idlefastfood,Eatventure,Are you looking to become a restaurant million...,https://play.google.com/store/apps/details?id=...,"GAME_SIMULATION,GAME",False
7477,com.alibaba.aliexpresshd,AliExpress,Maximum deals. Maximum fun. Shop our biggest s...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False


## Ask ChatGPT a question on every row

In [9]:
df["for_kids"] = df.apply(chatgpt_ask("Given the app description:\n{description}\n\nIs it for kids?\nAnswer only yes or no"), axis=1)
df

,bundle_id,title,description,store_url,category_names,ios,for_kids
36869,com.nexters.herowars,Hero Wars – Fantasy Battles,Unlock skills and battle enemies with Hero War...,https://play.google.com/store/apps/details?id=...,"GAME_ROLE_PLAYING,GAME",False,False
1021,1105855019,Gardenscapes,Welcome to Gardenscapes—the first hit from Pla...,https://apps.apple.com/us/app/gardenscapes/id1...,"Games,Entertainment,Puzzle,Simulation",True,False
2156,1200391796,June's Journey: Hidden Objects,I spy a mystery! Find the hidden objects to cr...,https://apps.apple.com/us/app/junes-journey-hi...,"Games,Adventure,Puzzle",True,False
60188,net.supertreat.solitaire,Solitaire Grand Harvest,Welcome to Solitaire Grand Harvest! Play this ...,https://play.google.com/store/apps/details?id=...,"GAME_CARD,GAME",False,False
659,air.com.playtika.slotomania,Slotomania™ Slots Casino Games,Slotomania wishes you a Happy New Year! Play n...,https://play.google.com/store/apps/details?id=...,"GAME_CASINO,GAME",False,False
34033,com.mistplay.mistplay,MISTPLAY: Play to earn rewards,"Introducing Mistplay, the app that lets you ea...",https://play.google.com/store/apps/details?id=...,"ENTERTAINMENT,APPLICATION",False,False
6673,892521917,Tiki Solitaire TriPeaks,Tiki Solitaire TriPeaks: the classic Solitaire...,https://apps.apple.com/us/app/tiki-solitaire-t...,"Games,Card,Puzzle",True,False
26290,com.hwqgrhhjfd.idlefastfood,Eatventure,Are you looking to become a restaurant million...,https://play.google.com/store/apps/details?id=...,"GAME_SIMULATION,GAME",False,False
7477,com.alibaba.aliexpresshd,AliExpress,Maximum deals. Maximum fun. Shop our biggest s...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,False


## Question:
Please rewrite the code such that the response to a yes/no question would be boolean (`True` or `False`)

**My first attempt:** 
Given your chatgpt_ask function's design, it returns a function (pandas_func) that is to be applied to a pandas DataFrame. To modify the code such that the response to a yes/no question is boolean (True or False), I add an additional step to process the output of chatgpt_ask.\\
Here's how I  do it:\\
- Apply the chatgpt_ask function: First, use the chatgpt_ask function to get 'yes'/'no' answers.
- Convert the Responses to Boolean: After getting the responses, convert them from 'yes'/'no' to boolean values.\\


**My second (and final) attempt:** 
I ensure that the chatgpt_ask function directly returns a boolean response based on a yes/no question, so I modify the chatgpt_ask function itself. This change will make the function more versatile and allow me to use it directly with df.apply without needing additional steps to convert the responses.

In [10]:
## MY FIRST ATTEMPT

# # Apply the chatgpt_ask function
# df["for_kids_response"] = df.apply(chatgpt_ask("Given the app description:\n{description}\n\nIs it for kids?\nAnswer only yes or no"), axis=1)

# # Function to convert 'yes'/'no' to True/False
# def response_to_bool(response):
#     if response is not None:
#         return response.lower().strip() == 'yes'
#     return False  # or return None if you prefer to keep non-responses as None

# # Convert 'yes'/'no' to True/False
# df["for_kids"] = df["for_kids_response"].apply(response_to_bool)
# df

